# Reversible Dimerization Example (Gillespie)

This notebook demonstrates:
1. A bounded reaction model: A + B <-> AB
2. Continuous-time sampling with Gillespie
3. Measurement before state modification
4. Basic analysis of the resulting trajectory

In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(path=joinpath(@__DIR__, ".."))
Pkg.instantiate()

using Random
using StatsBase
using MonteCarloX

## Reversible dimer model

In [ ]:
mutable struct ReversibleDimerModel <: AbstractSystem
    A::Int
    B::Int
    AB::Int
    k_on::Float64
    k_off::Float64
end

# Event 1: association A + B -> AB
# Event 2: dissociation AB -> A + B
reaction_rates(sys::ReversibleDimerModel, t::Real) = [
    sys.k_on * sys.A * sys.B,
    sys.k_off * sys.AB
]

function modify!(sys::ReversibleDimerModel, event::Int, t::Real)
    if event == 1
        if sys.A > 0 && sys.B > 0
            sys.A -= 1
            sys.B -= 1
            sys.AB += 1
        end
    elseif event == 2
        if sys.AB > 0
            sys.A += 1
            sys.B += 1
            sys.AB -= 1
        end
    end
    return sys
end

## Setup algorithm and measurements

In [ ]:
rng = MersenneTwister(23)
sys = ReversibleDimerModel(30, 30, 0, 0.01, 0.5)
alg = Gillespie(rng)

T = 200.0
measurement_times = collect(0.0:0.5:T)
measurements = Measurements([
    :A => (s -> s.A) => Int[],
    :B => (s -> s.B) => Int[],
    :AB => (s -> s.AB) => Int[]
], measurement_times)

measure!(measurements, sys, alg.time)
println("Initial state: A=$(sys.A), B=$(sys.B), AB=$(sys.AB)")

## Run continuous-time simulation
Measurement is evaluated after time advance and before state modification.
Time is stored in alg.time (no explicit time argument in next).

In [ ]:
while alg.time <= T
    dt, event = next(alg, t -> reaction_rates(sys, t))
    t_new = alg.time + dt
    measure!(measurements, sys, t_new)
    modify!(sys, event, t_new)
    alg.time = t_new
    alg.steps += 1
end

println("Final simulated time: $(round(alg.time, digits=3))")
println("Final state: A=$(sys.A), B=$(sys.B), AB=$(sys.AB)")
println("Events sampled: $(alg.steps)")

## Analyze trajectory

In [ ]:
A_traj = measurements[:A].data
B_traj = measurements[:B].data
AB_traj = measurements[:AB].data

println("Measured samples: $(length(AB_traj))")
println("Mean A:  $(round(mean(A_traj), digits=3))")
println("Mean B:  $(round(mean(B_traj), digits=3))")
println("Mean AB: $(round(mean(AB_traj), digits=3))")

In [ ]:
using Plots
plot(measurement_times, A_traj, xlabel="time", ylabel="count", label="A", lw=2)
plot!(measurement_times, B_traj, label="B", lw=2)
plot!(measurement_times, AB_traj, label="AB", lw=2)